In [0]:
library(dataiku)
library(rpart)
library(dplyr)
library(caret)
library(data.table)
library(mlflow)
library(reticulate)
library(Metrics)

In [0]:
# Recipe inputs
truncated_train <- dkuReadDataset("truncated_train", samplingMethod="head", nbRows=100000)
truncated_validation <- dkuReadDataset("truncated_validation", samplingMethod="head", nbRows=100000)

In [0]:
# Fitting tree for wind and rain
# wind_max prediction using decision trees

trunc_wind_model <- rpart(wind_max ~ track_min_dist, 
                       data = truncated_train, 
                       method = "anova")

trunc_rain_model <- rpart(rain_total ~ track_min_dist, 
                       data = truncated_train, 
                       method = "anova")

In [0]:
# add the predictions of wind and rainfall to the dataframes
df_trunc_train <- truncated_train %>%
  mutate(wind_max_pred = predict(trunc_wind_model, 
                         newdata = truncated_train), 
         rain_total_pred = predict(trunc_rain_model, 
                         newdata = truncated_train)
        )

trunc_damage_fit_reg <- rpart(damage_perc ~ wind_max_pred + 
                           rain_total_pred + 
                           roof_strong_wall_strong + 
                           roof_strong_wall_light +
                           roof_strong_wall_salv + 
                           roof_light_wall_strong + 
                           roof_light_wall_light + 
                           roof_light_wall_salv + 
                           roof_salv_wall_strong +
                           roof_salv_wall_light +
                           roof_salv_wall_salv +
                           ls_risk_pct +
                           ss_risk_pct +
                           ruggedness_mean +
                           slope_mean +
                           wind_blue_ss +
                           wind_yellow_ss +          
                           wind_orange_ss +          
                           wind_red_ss +
                           rain_blue_ss +
                           rain_yellow_ss +
                           rain_orange_ss +
                           rain_red_ss, 
                         method = "anova", 
                         data = df_high_train)

In [0]:
# Recipe outputs
trunk_scm_min_model <- dkuManagedFolderPath("dL4i4SKb")